In [2]:
from pathlib import Path
path_to_data_notebook = Path.cwd().parent.parent / "Data" / "1_Preprocesed_data.ipynb"
%run {path_to_data_notebook}

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import datetime 
import time
from sklearn.model_selection import train_test_split

2025-03-20 19:07:42.229585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742494062.248416   92525 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742494062.254039   92525 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-20 19:07:42.272419: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data

In [3]:
size = 40
x_range_to_cut = 0.001
df = get_whole_preprocessed_dataframe(size, x_range_to_cut)
# choosen property for y:
choosen_property_1 = "Total_surface_area[m2/g]"

df_with_data = df[df[choosen_property_1].notna()]    # get data if property are present
df_without_data = df[df[choosen_property_1].isna()]    # get data if property arent present

X = df_with_data['processed_flatten_isotherm_X_Y'] # x = 'processed_flatten_isotherm_X_Y'
y = df_with_data # y - temporary all columns for prediction inspection, in next steps we extract only choosen property

X_train, X_test, y_train_nested, y_test_nested, = train_test_split(X, y, test_size=0.02, random_state=42) #random_state=42

X_train = X_train.tolist()
X_test = X_test.tolist()
y_train = y_train_nested[choosen_property_1].to_numpy()
y_test = y_test_nested[choosen_property_1].to_numpy() 


scaler = StandardScaler() # only for x data
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 

print(f'X_train size = {len(X_train)}')
print(f'y_train size = {len(X_train)}')
print(f'X_test size = {len(X_test)}')
print(f'y_test size = {len(X_test)}')

X_without = df_without_data['processed_flatten_isotherm_X_Y'].tolist()
X_without = scaler.transform(X_without) 
y_without = df_without_data[choosen_property_1].to_numpy()
print(f'X_without size = {len(X_without)}')
print(f'y_without size = {len(y_without)}')

/home/pc/moje/python_projects/adsorbents_isotherms/Data/all_data.csv
------------------------------- Preprocessing_data start -------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Article_name                    1011 non-null   object 
 1   Figure_number                   1011 non-null   object 
 2   Sample_name                     1011 non-null   object 
 3   Total_surface_area[m2/g]        928 non-null    float64
 4   Total_pore_volume[cm3/g]        778 non-null    float64
 5   Micropore_volume[cm3/g]         730 non-null    float64
 6   Mesopore_volume[cm3/g]          21 non-null     float64
 7   Average_pore_diameter[nm]       121 non-null    float64
 8   Impregnation_ratio[agent/char]  20 non-null     float64
 9   Activation_type                 33 non-null     object 
 10

X_without size = 82
y_without size = 82


# MLP model:

In [20]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

def calculate_time(transcription_time):
    timedelta_obj = datetime.timedelta(seconds=int(transcription_time))
    return str(timedelta_obj)
epochs_number = 500
model = tf.keras.Sequential([ 
    tf.keras.layers.Dense(size*4, activation='relu'), 
    tf.keras.layers.Dense(size*3, activation='relu'),
    tf.keras.layers.Dense(size*3, activation='relu'),
    tf.keras.layers.Dense(1)                      
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=['mae', 'mse']
model.compile(optimizer=optimizer, loss='mse', metrics=metrics)
# model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# history  =  model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=2)
start = time.time()
history  =  model.fit(X_train, y_train, epochs=epochs_number, batch_size=32, validation_split=0.1, verbose=2)
end = time.time()
transcription_time = calculate_time(round((end - start),2))
print(f"Model training time: "+transcription_time)
model.summary()
results = model.evaluate(X_test, y_test, verbose=0)
print("Metrics:",results)
for index, metric in enumerate(metrics):
    print(f"{metric} = {results[index+1]}")


Num GPUs Available:  1
Num GPUs Available:  1
Epoch 1/500
25/25 - 2s - 78ms/step - loss: 1555085.2500 - mae: 1050.3359 - mse: 1555085.2500 - val_loss: 1500277.5000 - val_mae: 1046.6173 - val_mse: 1500277.5000
Epoch 2/500
25/25 - 0s - 4ms/step - loss: 1134892.5000 - mae: 885.9892 - mse: 1134892.5000 - val_loss: 534192.5625 - val_mae: 616.0380 - val_mse: 534192.5625
Epoch 3/500
25/25 - 0s - 4ms/step - loss: 291226.7500 - mae: 437.4803 - mse: 291226.7500 - val_loss: 270213.5312 - val_mae: 395.0743 - val_mse: 270213.5312
Epoch 4/500
25/25 - 0s - 4ms/step - loss: 199470.6406 - mae: 367.6594 - mse: 199470.6406 - val_loss: 232432.8438 - val_mae: 361.2091 - val_mse: 232432.8438
Epoch 5/500
25/25 - 0s - 4ms/step - loss: 177565.8906 - mae: 336.7212 - mse: 177565.8906 - val_loss: 204399.5000 - val_mae: 349.9569 - val_mse: 204399.5000
Epoch 6/500
25/25 - 0s - 4ms/step - loss: 162193.1250 - mae: 320.6590 - mse: 162193.1250 - val_loss: 190971.9844 - val_mae: 331.8990 - val_mse: 190971.9844
Epoch 7/5

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 160)            │        12,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 120)            │        19,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140,765 (549.87 KB)

 Trainable params: 46,921 (183.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 93,844 (366.58 KB)

Metrics: [10793.4775390625, 77.79916381835938, 10793.4775390625]
mae = 77.79916381835938
mse = 10793.4775390625


In [21]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
 przewidywana --> [1511.3657]     1540.0 <-- rzeczyswista     blad: [28.63427734]   Data132 3 C-600
 przewidywana --> [977.2134]     987.0 <-- rzeczyswista     blad: [9.78662109]   Data75 4_3 1-25h
 przewidywana --> [780.91077]     652.6 <-- rzeczyswista     blad: [128.3107666]   Data91 3_1 AC-KG
 przewidywana --> [1072.3789]     1218.62 <-- rzeczyswista     blad: [146.24109375]   Data134 1 AC_15-400-1
 przewidywana --> [551.13794]     659.0 <-- rzeczyswista     blad: [107.86206055]   Data7 3 RK-600
 przewidywana --> [92.941734]     24.5399 <-- rzeczyswista     blad: [68.40183431]   Data53 1_6 ZCac
 przewidywana --> [1016.02637]     947.6 <-- rzeczyswista     blad: [68.42636719]   Data54 3 CZ0.40
 przewidywana --> [588.0094]     640.0 <-- rzeczyswista     blad: [51.99060059]   Data60 1 500
 przewidywana --> [1016.0777]     985.3 <-- rzeczyswista     blad: [30.77769775]   Data43 6 F1-8-61
 przewidywana --> [1750.4131]     1784.0 <-- rzeczyswista   

# Regresja liniowa

In [22]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [23]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1259.2971258496586     1540.0 <-- rzeczyswista     blad: 280.70287415034136   Data132 3 C-600
 przewidywana --> 970.3294317599833     987.0 <-- rzeczyswista     blad: 16.670568240016678   Data75 4_3 1-25h
 przewidywana --> 840.2760102867762     652.6 <-- rzeczyswista     blad: 187.6760102867762   Data91 3_1 AC-KG
 przewidywana --> 1231.4955206755883     1218.62 <-- rzeczyswista     blad: 12.875520675588405   Data134 1 AC_15-400-1
 przewidywana --> 553.960903276098     659.0 <-- rzeczyswista     blad: 105.03909672390205   Data7 3 RK-600
 przewidywana --> -196.06649632252174     24.5399 <-- rzeczyswista     blad: 220.60639632252173   Data53 1_6 ZCac
 przewidywana --> 850.8351987410724     947.6 <-- rzeczyswista     blad: 96.7648012589276   Data54 3 CZ0.40
 przewidywana --> 597.9034801741611     640.0 <-- rzeczyswista     blad: 42.09651982583887   Data60 1 500
 przewidywana --> 1035.8613212318849     985.3 <-- rzeczyswista     blad: 50.561321231884904   Data43 6 F1-8-61


# SGDRegressor

In [31]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor(max_iter=10000, tol=1e-5, penalty=None, eta0=0.01, n_iter_no_change=100, random_state=42)
model.fit(X_train, y_train.ravel())  # y.ravel() because fit() expects 1D targets

SGDRegressor(max_iter=10000, n_iter_no_change=100, penalty=None,
             random_state=42, tol=1e-05)

In [32]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1361.8502198926153     1540.0 <-- rzeczyswista     blad: 178.1497801073847   Data132 3 C-600
 przewidywana --> 962.4227218672399     987.0 <-- rzeczyswista     blad: 24.577278132760057   Data75 4_3 1-25h
 przewidywana --> 853.8593897645036     652.6 <-- rzeczyswista     blad: 201.25938976450357   Data91 3_1 AC-KG
 przewidywana --> 1205.944998978668     1218.62 <-- rzeczyswista     blad: 12.67500102133181   Data134 1 AC_15-400-1
 przewidywana --> 593.5089202025842     659.0 <-- rzeczyswista     blad: 65.49107979741575   Data7 3 RK-600
 przewidywana --> -217.47820895863651     24.5399 <-- rzeczyswista     blad: 242.0181089586365   Data53 1_6 ZCac
 przewidywana --> 842.8571595774175     947.6 <-- rzeczyswista     blad: 104.7428404225825   Data54 3 CZ0.40
 przewidywana --> 652.8168840560793     640.0 <-- rzeczyswista     blad: 12.816884056079289   Data60 1 500
 przewidywana --> 1054.4947372790905     985.3 <-- rzeczyswista     blad: 69.19473727909053   Data43 6 F1-8-61
 p

# Ridge

In [33]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=0.1, solver="cholesky")
model.fit(X_train, y_train)

Ridge(alpha=0.1, solver='cholesky')

In [34]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1277.00420667586     1540.0 <-- rzeczyswista     blad: 262.9957933241401   Data132 3 C-600
 przewidywana --> 953.5867314180604     987.0 <-- rzeczyswista     blad: 33.41326858193963   Data75 4_3 1-25h
 przewidywana --> 842.9028856691059     652.6 <-- rzeczyswista     blad: 190.30288566910588   Data91 3_1 AC-KG
 przewidywana --> 1224.5992752123614     1218.62 <-- rzeczyswista     blad: 5.979275212361472   Data134 1 AC_15-400-1
 przewidywana --> 564.5079526234954     659.0 <-- rzeczyswista     blad: 94.49204737650462   Data7 3 RK-600
 przewidywana --> -269.97444583793845     24.5399 <-- rzeczyswista     blad: 294.51434583793844   Data53 1_6 ZCac
 przewidywana --> 823.1928130454351     947.6 <-- rzeczyswista     blad: 124.40718695456496   Data54 3 CZ0.40
 przewidywana --> 628.7242743919162     640.0 <-- rzeczyswista     blad: 11.275725608083803   Data60 1 500
 przewidywana --> 1069.8179700149635     985.3 <-- rzeczyswista     blad: 84.51797001496357   Data43 6 F1-8-61
 p

# Lasso

In [35]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.1)
model.fit(X_train, y_train)

/home/pc/.cache/pypoetry/virtualenvs/adorbents-isotherms-FmcCsTt1-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.977e+06, tolerance: 3.977e+04
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [36]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1309.8721748659884     1540.0 <-- rzeczyswista     blad: 230.12782513401157   Data132 3 C-600
 przewidywana --> 950.3698682868496     987.0 <-- rzeczyswista     blad: 36.63013171315038   Data75 4_3 1-25h
 przewidywana --> 850.910299601327     652.6 <-- rzeczyswista     blad: 198.31029960132696   Data91 3_1 AC-KG
 przewidywana --> 1213.034203380883     1218.62 <-- rzeczyswista     blad: 5.585796619116991   Data134 1 AC_15-400-1
 przewidywana --> 578.4153697164966     659.0 <-- rzeczyswista     blad: 80.58463028350343   Data7 3 RK-600
 przewidywana --> -280.02609371407016     24.5399 <-- rzeczyswista     blad: 304.56599371407015   Data53 1_6 ZCac
 przewidywana --> 830.6112126693729     947.6 <-- rzeczyswista     blad: 116.98878733062713   Data54 3 CZ0.40
 przewidywana --> 649.1491905303142     640.0 <-- rzeczyswista     blad: 9.149190530314172   Data60 1 500
 przewidywana --> 1073.428950330479     985.3 <-- rzeczyswista     blad: 88.12895033047903   Data43 6 F1-8-61
 pr

# ElasticNet

In [37]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=0.1, l1_ratio=0.5)
model.fit(X_train, y_train)

/home/pc/.cache/pypoetry/virtualenvs/adorbents-isotherms-FmcCsTt1-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.629e+06, tolerance: 3.977e+04
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.1)

In [38]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1409.391719085149     1540.0 <-- rzeczyswista     blad: 130.6082809148511   Data132 3 C-600
 przewidywana --> 943.5430572067556     987.0 <-- rzeczyswista     blad: 43.456942793244366   Data75 4_3 1-25h
 przewidywana --> 856.2090498567795     652.6 <-- rzeczyswista     blad: 203.6090498567795   Data91 3_1 AC-KG
 przewidywana --> 1229.1586517426476     1218.62 <-- rzeczyswista     blad: 10.538651742647744   Data134 1 AC_15-400-1
 przewidywana --> 582.5363793087605     659.0 <-- rzeczyswista     blad: 76.4636206912395   Data7 3 RK-600
 przewidywana --> -98.92856005980775     24.5399 <-- rzeczyswista     blad: 123.46846005980775   Data53 1_6 ZCac
 przewidywana --> 893.2149167214793     947.6 <-- rzeczyswista     blad: 54.38508327852071   Data54 3 CZ0.40
 przewidywana --> 671.3303714082128     640.0 <-- rzeczyswista     blad: 31.33037140821284   Data60 1 500
 przewidywana --> 1022.2032172003424     985.3 <-- rzeczyswista     blad: 36.9032172003424   Data43 6 F1-8-61
 prze

# DecisionTreeRegressor

In [39]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=2, random_state=42)
model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [40]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1032.4254142011832     1540.0 <-- rzeczyswista     blad: 507.57458579881677   Data132 3 C-600
 przewidywana --> 1032.4254142011832     987.0 <-- rzeczyswista     blad: 45.42541420118323   Data75 4_3 1-25h
 przewidywana --> 462.7680097178683     652.6 <-- rzeczyswista     blad: 189.8319902821317   Data91 3_1 AC-KG
 przewidywana --> 1032.4254142011832     1218.62 <-- rzeczyswista     blad: 186.19458579881666   Data134 1 AC_15-400-1
 przewidywana --> 462.7680097178683     659.0 <-- rzeczyswista     blad: 196.23199028213168   Data7 3 RK-600
 przewidywana --> 462.7680097178683     24.5399 <-- rzeczyswista     blad: 438.22810971786834   Data53 1_6 ZCac
 przewidywana --> 1032.4254142011832     947.6 <-- rzeczyswista     blad: 84.82541420118321   Data54 3 CZ0.40
 przewidywana --> 462.7680097178683     640.0 <-- rzeczyswista     blad: 177.23199028213168   Data60 1 500
 przewidywana --> 1032.4254142011832     985.3 <-- rzeczyswista     blad: 47.125414201183276   Data43 6 F1-8-6

# KNeighborsRegressor

In [41]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor( n_neighbors=2 )

# Train the model
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=2)

In [42]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1440.5     1540.0 <-- rzeczyswista     blad: 99.5   Data132 3 C-600
 przewidywana --> 981.5     987.0 <-- rzeczyswista     blad: 5.5   Data75 4_3 1-25h
 przewidywana --> 790.045     652.6 <-- rzeczyswista     blad: 137.44499999999994   Data91 3_1 AC-KG
 przewidywana --> 1164.87     1218.62 <-- rzeczyswista     blad: 53.75   Data134 1 AC_15-400-1
 przewidywana --> 643.0     659.0 <-- rzeczyswista     blad: 16.0   Data7 3 RK-600
 przewidywana --> 163.61385     24.5399 <-- rzeczyswista     blad: 139.07395000000002   Data53 1_6 ZCac
 przewidywana --> 956.85     947.6 <-- rzeczyswista     blad: 9.25   Data54 3 CZ0.40
 przewidywana --> 631.5     640.0 <-- rzeczyswista     blad: 8.5   Data60 1 500
 przewidywana --> 940.5     985.3 <-- rzeczyswista     blad: 44.799999999999955   Data43 6 F1-8-61
 przewidywana --> 2021.0     1784.0 <-- rzeczyswista     blad: 237.0   Data27 1 KUA21701-2
 przewidywana --> 1261.5     1305.0 <-- rzeczyswista     blad: 43.5   Data27 1 KUA21701-200


# LinearSVR

In [43]:
from sklearn.svm import LinearSVR

model = LinearSVR(epsilon=0.5, dual=True, random_state=42)
model.fit(X_train, y_train)


LinearSVR(dual=True, epsilon=0.5, random_state=42)

In [44]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1143.2527546481351     1540.0 <-- rzeczyswista     blad: 396.7472453518649   Data132 3 C-600
 przewidywana --> 740.4729884625933     987.0 <-- rzeczyswista     blad: 246.52701153740668   Data75 4_3 1-25h
 przewidywana --> 629.786501922971     652.6 <-- rzeczyswista     blad: 22.813498077029067   Data91 3_1 AC-KG
 przewidywana --> 972.3364372008195     1218.62 <-- rzeczyswista     blad: 246.28356279918034   Data134 1 AC_15-400-1
 przewidywana --> 419.13703807373156     659.0 <-- rzeczyswista     blad: 239.86296192626844   Data7 3 RK-600
 przewidywana --> -86.5299337948768     24.5399 <-- rzeczyswista     blad: 111.0698337948768   Data53 1_6 ZCac
 przewidywana --> 751.7862097471628     947.6 <-- rzeczyswista     blad: 195.81379025283718   Data54 3 CZ0.40
 przewidywana --> 528.3668459389012     640.0 <-- rzeczyswista     blad: 111.63315406109882   Data60 1 500
 przewidywana --> 796.1399322731282     985.3 <-- rzeczyswista     blad: 189.16006772687172   Data43 6 F1-8-61
 

# GradientBoostingRegressor

In [45]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
model.fit(X_train, y_train)


GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

In [46]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1194.5586951656187     1540.0 <-- rzeczyswista     blad: 345.4413048343813   Data132 3 C-600
 przewidywana --> 1002.8158964490627     987.0 <-- rzeczyswista     blad: 15.815896449062734   Data75 4_3 1-25h
 przewidywana --> 504.4485735344963     652.6 <-- rzeczyswista     blad: 148.15142646550373   Data91 3_1 AC-KG
 przewidywana --> 1002.8158964490627     1218.62 <-- rzeczyswista     blad: 215.80410355093716   Data134 1 AC_15-400-1
 przewidywana --> 433.1584919657491     659.0 <-- rzeczyswista     blad: 225.84150803425092   Data7 3 RK-600
 przewidywana --> 91.7484755486015     24.5399 <-- rzeczyswista     blad: 67.2085755486015   Data53 1_6 ZCac
 przewidywana --> 1002.8158964490627     947.6 <-- rzeczyswista     blad: 55.21589644906271   Data54 3 CZ0.40
 przewidywana --> 433.1584919657491     640.0 <-- rzeczyswista     blad: 206.84150803425092   Data60 1 500
 przewidywana --> 1002.8158964490627     985.3 <-- rzeczyswista     blad: 17.51589644906278   Data43 6 F1-8-61
 

# HistGradientBoostingRegressor

In [47]:
from sklearn.ensemble import HistGradientBoostingRegressor

model =  HistGradientBoostingRegressor(categorical_features=[0], random_state=42)
model.fit(X_train, y_train)


HistGradientBoostingRegressor(categorical_features=[0], random_state=42)

In [48]:
predictions_test = model.predict(X_test)
y_test_for_compare = y_test
y_dataframe = y_test_nested

error_threshold = 30
max_error_threshold = 30

list_below =[]
list_above=[]
errors = []
for i in range(len(predictions_test)):
    data_name = y_dataframe.iloc[i,0]
    graph_number = y_dataframe.iloc[i,1]
    sample_name = y_dataframe.iloc[i,2]
    error = abs(predictions_test[i] - y_test_for_compare[i])
    
    errors.append(error)
    if(error_threshold<=error):
        list_above.append(i)
    else: list_below.append(i)

    print(f" przewidywana --> {predictions_test[i]}     {y_test_for_compare[i]} <-- rzeczyswista     blad: {error}   {data_name} {graph_number} {sample_name}")

max_error_index = errors.index(max(errors))
max_error_data_name = y_dataframe.iloc[max_error_index,0]
max_error_graph_number = y_dataframe.iloc[max_error_index,1]
max_error_sample_name = y_dataframe.iloc[max_error_index,2]
text_to_display = f"Max error: {max(errors)}  dla {max_error_data_name} {max_error_graph_number} {max_error_sample_name}  "  + "\n" + f"for error treshold[{error_threshold}] - {round((len(list_below)/len(predictions_test))*100,2)}%     {len(list_below)} izotermy na {len(predictions_test)}"
print(text_to_display)


 przewidywana --> 1436.4809186443606     1540.0 <-- rzeczyswista     blad: 103.5190813556394   Data132 3 C-600
 przewidywana --> 963.7035451262338     987.0 <-- rzeczyswista     blad: 23.29645487376615   Data75 4_3 1-25h
 przewidywana --> 711.3564360027749     652.6 <-- rzeczyswista     blad: 58.756436002774876   Data91 3_1 AC-KG
 przewidywana --> 1151.7616367559742     1218.62 <-- rzeczyswista     blad: 66.85836324402567   Data134 1 AC_15-400-1
 przewidywana --> 570.7206891383107     659.0 <-- rzeczyswista     blad: 88.27931086168928   Data7 3 RK-600
 przewidywana --> 128.01032673706592     24.5399 <-- rzeczyswista     blad: 103.47042673706592   Data53 1_6 ZCac
 przewidywana --> 887.7560499187214     947.6 <-- rzeczyswista     blad: 59.84395008127865   Data54 3 CZ0.40
 przewidywana --> 632.7925330658858     640.0 <-- rzeczyswista     blad: 7.207466934114223   Data60 1 500
 przewidywana --> 1062.1292794731569     985.3 <-- rzeczyswista     blad: 76.82927947315693   Data43 6 F1-8-61
 pr